# Beam splitter model

Use the multihead (2-head) gates in the phase space 
to create a network that represent a coherent state with with a beam splitter

The state propagate in a complex medium

<img src="../img/logo_circular.png" width="20" height="20" />@by claudio<br>


nonlinearxwaves@gmail.com<br>
@created 20 december 2020<br>
@version 21 sept 2023

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # disable warning messages 

In [2]:
import numpy as np
from thqml import phasespace as ps
from thqml.utilities import utilities
import tensorflow as tf
#import tensorflow_addons as tfa
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
tf_complex = tf.complex
tf_real = tf.float32
np_complex = complex
np_real = np.float64

In [4]:
tf.keras.backend.clear_session()

In [5]:
np.set_printoptions(precision=2)

## Dimension (number of modes times 2)

In [6]:
N = 10

## Index of the mode 1 (between 0 an N/2)

In [7]:
n_0=0

## Index of the mode 2 

In [8]:
n_1=3

## Displacer (not used)

In [9]:
dtarget=3.2*np.ones((N,1))
norm = np.linalg.norm(dtarget)
dtarget = dtarget /norm
displacer = ps.DisplacementLayerConstant(dtarget)

## Build vacuum by the Gaussian state

In [10]:
vacuum = ps.VacuumLayer(N)

## Two mode squeezer

In [11]:
theta_np=np.pi/4;

In [12]:
bs=ps.BeamSplitterLayer(N, theta=theta_np, n_0=n_0, n_1=n_1, phi0=0, phi1=0 )

In [13]:
n_1

3

In [14]:
bs.paddings11

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[0, 8],
       [0, 8]], dtype=int32)>

In [15]:
Ms, MsI=bs.get_M(); Ms_np = Ms.numpy();print(Ms_np)

[[ 0.71  0.    0.    0.    0.    0.   -0.71  0.    0.    0.  ]
 [ 0.    0.71  0.    0.    0.    0.    0.   -0.71  0.    0.  ]
 [ 0.    0.    1.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    1.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    1.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    1.    0.    0.    0.    0.  ]
 [ 0.71  0.    0.    0.    0.    0.    0.71  0.    0.    0.  ]
 [ 0.    0.71  0.    0.    0.    0.    0.    0.71  0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    1.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    1.  ]]


### Check if the matrix is symplectic

In [16]:
utilities.printonscreennp(np.matmul(Ms.numpy(),MsI.numpy()))

+1.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i -0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i 
+0.0+0.0i +1.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i -0.0+0.0i +0.0+0.0i +0.0+0.0i 
+0.0+0.0i +0.0+0.0i +1.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i 
+0.0+0.0i +0.0+0.0i +0.0+0.0i +1.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i 
+0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +1.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i 
+0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +1.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i 
-0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +1.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i 
+0.0+0.0i -0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +1.0+0.0i +0.0+0.0i +0.0+0.0i 
+0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +1.0+0.0i +0.0+0.0i 
+0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +

The previous matrix must be unitary

# Build the model

In [17]:
xin = tf.keras.layers.Input(N)
x1, d1 = bs(xin)
chir, chii = vacuum(x1, d1)
bs = tf.keras.Model(inputs = xin, outputs=[chir, chii])

# Evaluate the covariance matrix

In [18]:
cov_layer = ps.covariance(N)
covariance_matrix, mean_R, _ = cov_layer(chir,chii, bs)
bs_cov = tf.keras.Model(inputs = xin, outputs=[covariance_matrix, mean_R])

# Training points

In [19]:
xtrain = np.random.rand(1, N)-0.5

# Covariance and displacement

In [20]:
cov0,d0=bs_cov(xtrain); 

## Displacement

In [21]:
print(d0)

tf.Tensor([[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(1, 10), dtype=float32)


## Eigenvalues and eigenvectors of the covariance matrix

In [22]:
eigs, eigv=np.linalg.eig(cov0.numpy()); print(eigs); print(eigv)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[[ 0.71 -0.71  0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.   -0.71  0.71  0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    1.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    1.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    1.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    1.    0.    0.  ]
 [-0.71 -0.71  0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.71  0.71  0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    1.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    1.  ]]


Note that the BS rotate the eigenvectors to (x1+x2), (x1-x1), (p1+p2), and (p1-p2) whent it a 50:50 with theta=pi/4

Note that eigenvalues are all unitaries and we have a coherent state because no squeezing is present